In [55]:
import io
from typing import Iterable, Callable
import zipfile
import traceback
from dataclasses import dataclass

import requests


@dataclass
class RawRepositoryFile:
    filename: str
    content: str


class GithubRepositoryDataReader:
    """
    Downloads and parses markdown and code files from a GitHub repository.
    """

    def __init__(self,
                 repo_owner: str,
                 repo_name: str,
                 allowed_extensions: Iterable[str] | None = None,
                 filename_filter: Callable[[str], bool] | None = None
                 ):
        """
        Initialize the GitHub repository data reader.

        Args:
            repo_owner: The owner/organization of the GitHub repository
            repo_name: The name of the GitHub repository
            allowed_extensions: Optional set of file extensions to include
                    (e.g., {"md", "py"}). If not provided, all file types are included
            filename_filter: Optional callable to filter files by their path
        """
        prefix = "https://codeload.github.com"
        self.url = (
            f"{prefix}/{repo_owner}/{repo_name}/zip/refs/heads/main"
        )

        if allowed_extensions is not None:
            self.allowed_extensions = {ext.lower() for ext in allowed_extensions}

        if filename_filter is None:
            self.filename_filter = lambda filepath: True
        else:
            self.filename_filter = filename_filter

    def read(self) -> list[RawRepositoryFile]:
        """
        Download and extract files from the GitHub repository.

        Returns:
            List of RawRepositoryFile objects for each processed file

        Raises:
            Exception: If the repository download fails
        """
        resp = requests.get(self.url)
        if resp.status_code != 200:
            raise Exception(f"Failed to download repository: {resp.status_code}")

        zf = zipfile.ZipFile(io.BytesIO(resp.content))
        repository_data = self._extract_files(zf)
        zf.close()

        return repository_data

    def _extract_files(self, zf: zipfile.ZipFile) -> list[RawRepositoryFile]:
        """
        Extract and process files from the zip archive.

        Args:
            zf: ZipFile object containing the repository data

        Returns:
            List of RawRepositoryFile objects for each processed file
        """
        data = []

        for file_info in zf.infolist():
            filepath = self._normalize_filepath(file_info.filename)

            if self._should_skip_file(filepath):
                continue

            try:
                with zf.open(file_info) as f_in:
                    content = f_in.read().decode("utf-8", errors="ignore")
                    if content is not None:
                        content = content.strip()

                    file = RawRepositoryFile(
                        filename=filepath,
                        content=content
                    )
                    data.append(file)

            except Exception as e:
                print(f"Error processing {file_info.filename}: {e}")
                traceback.print_exc()
                continue

        return data

    def _should_skip_file(self, filepath: str) -> bool:
        """
        Determine whether a file should be skipped during processing.

        Args:
            filepath: The file path to check

        Returns:
            True if the file should be skipped, False otherwise
        """
        filepath = filepath.lower()

        # directory
        if filepath.endswith("/"):
            return True

        # hidden file
        filename = filepath.split("/")[-1]
        if filename.startswith("."):
            return True

        if self.allowed_extensions:
            ext = self._get_extension(filepath)
            if ext not in self.allowed_extensions:
                return True

        if not self.filename_filter(filepath):
            return True

        return False

    def _get_extension(self, filepath: str) -> str:
        """
        Extract the file extension from a filepath.

        Args:
            filepath: The file path to extract extension from

        Returns:
            The file extension (without dot) or empty string if no extension
        """
        filename = filepath.lower().split("/")[-1]
        if "." in filename:
            return filename.rsplit(".", maxsplit=1)[-1]
        else:
            return ""

    def _normalize_filepath(self, filepath: str) -> str:
        """
        Removes the top-level directory from the file path inside the zip archive.
        'repo-main/path/to/file.py' -> 'path/to/file.py'

        Args:
            filepath: The original filepath from the zip archive

        Returns:
            The normalized filepath with top-level directory removed
        """
        parts = filepath.split("/", maxsplit=1)
        if len(parts) > 1:
            return parts[1]
        else:
            return parts[0]


In [56]:
def read_github_data():
    allowed_extensions = {"md", "mdx"}

    repo_owner = 'evidentlyai'
    repo_name = 'docs'

    reader = GithubRepositoryDataReader(
        repo_owner,
        repo_name,
        allowed_extensions=allowed_extensions
    )

    return reader.read()


In [57]:
data_raw = read_github_data()
print(f"Downloaded {len(data_raw)} files")

Downloaded 95 files


In [58]:
data_raw[15]

RawRepositoryFile(filename='docs/library/tags_metadata.mdx', content='---\ntitle: \'Add tags and metadata\'\ndescription: \'How to add metadata to evaluations.\'\n---\n\nThis is relevant when you logging Reports to the Platform. Tags help you associate each Report with a specific model / prompt version, time period, or other context.\n\n## Add timestamp\n\nEach Report run has a single timestamp. By default, Evidently assigns `datetime.now()` as the run time based on the user\'s time zone.\n\nYou can also specify a custom timestamp by passing it to the `run()` method:\n\n```python\nfrom datetime import datetime\n\nmy_eval_4 = report.run(eval_data_1,\n                       eval_data_2,\n                       timestamp=datetime(2024, 1, 29))\n```\n\nBecause timestamps are fully customizable, you can log Reports asynchronously or with a delay. For example, make an evaluation after receiving ground truth and backdate Reports to the relevant time period.\n\n## Add tags and metadata\n\nYou 

In [59]:
"""
Document chunking utilities for splitting large documents into smaller, overlapping pieces.

This module provides functionality to break down documents into chunks using a sliding
window approach, which is useful for processing large texts in smaller, manageable pieces
while maintaining context through overlapping content.
"""

from typing import Any, Dict, Iterable, List


def sliding_window(
        seq: Iterable[Any],
        size: int,
        step: int
    ) -> List[Dict[str, Any]]:
    """
    Create overlapping chunks from a sequence using a sliding window approach.

    Args:
        seq: The input sequence (string or list) to be chunked.
        size (int): The size of each chunk/window.
        step (int): The step size between consecutive windows.

    Returns:
        list: A list of dictionaries, each containing:
            - 'start': The starting position of the chunk in the original sequence
            - 'content': The chunk content

    Raises:
        ValueError: If size or step are not positive integers.

    Example:
        >>> sliding_window("hello world", size=5, step=3)
        [{'start': 0, 'content': 'hello'}, {'start': 3, 'content': 'lo wo'}]
    """
    if size <= 0 or step <= 0:
        raise ValueError("size and step must be positive")

    n = len(seq)
    result = []
    for i in range(0, n, step):
        batch = seq[i:i+size]
        result.append({'start': i, 'content': batch})
        if i + size > n:
            break

    return result


def chunk_documents(
        documents: Iterable[Dict[str, str]],
        size: int = 2000,
        step: int = 1000,
        content_field_name: str = 'content'
) -> List[Dict[str, str]]:
    """
    Split a collection of documents into smaller chunks using sliding windows.

    Takes documents and breaks their content into overlapping chunks while preserving
    all other document metadata (filename, etc.) in each chunk.

    Args:
        documents: An iterable of document dictionaries. Each document must have a content field.
        size (int, optional): The maximum size of each chunk. Defaults to 2000.
        step (int, optional): The step size between chunks. Defaults to 1000.
        content_field_name (str, optional): The name of the field containing document content.
                                          Defaults to 'content'.

    Returns:
        list: A list of chunk dictionaries. Each chunk contains:
            - All original document fields except the content field
            - 'start': Starting position of the chunk in original content
            - 'content': The chunk content

    Example:
        >>> documents = [{'content': 'long text...', 'filename': 'doc.txt'}]
        >>> chunks = chunk_documents(documents, size=100, step=50)
        >>> # Or with custom content field:
        >>> documents = [{'text': 'long text...', 'filename': 'doc.txt'}]
        >>> chunks = chunk_documents(documents, content_field_name='text')
    """
    results = []

    for doc in documents:
        doc_copy = doc.copy()
        doc_content = doc_copy.pop(content_field_name)
        chunks = sliding_window(doc_content, size=size, step=step)
        for chunk in chunks:
            chunk.update(doc_copy)
        results.extend(chunks)

    return results

In [60]:
"""
Document indexing utilities for creating searchable indexes from document collections.

This module provides functionality to index documents using minsearch, with optional
chunking support for handling large documents.
"""

from minsearch import Index

def index_documents(documents, chunk: bool = False, chunking_params=None) -> Index:
    """
    Create a searchable index from a collection of documents.

    Args:
        documents: A collection of document dictionaries, each containing at least
                  'content' and 'filename' fields.
        chunk (bool, optional): Whether to chunk documents before indexing.
                               Defaults to False.
        chunking_params (dict, optional): Parameters for document chunking.
                                        Defaults to {'size': 2000, 'step': 1000}.
                                        Only used when chunk=True.

    Returns:
        Index: A fitted minsearch Index object ready for searching.

    Example:
        >>> docs = [{'content': 'Hello world', 'filename': 'doc1.txt'}]
        >>> index = index_documents(docs)
        >>> results = index.search('hello')
    """
    if chunk:
        if chunking_params is None:
            chunking_params = {'size': 2000, 'step': 1000}
        documents = chunk_documents(documents, **chunking_params)

    index = Index(
        text_fields=["content", "filename"],
    )

    index.fit(documents)
    return index

In [61]:
!uv add python-frontmatter
!uv add rich

Resolved 154 packages in 10ms
Audited 135 packages in 0.07ms
Resolved 154 packages in 0.74ms
Audited 135 packages in 0.03ms


In [62]:
import frontmatter
from typing import List, Dict, Any
from rich.progress import track

def parse_data(data_raw: List[RawRepositoryFile]) -> List[Dict[str, Any]]:
    print("📄 [bold blue]Parsing documents...[/bold blue]")

    data_parsed = []
    for f in track(data_raw, description="Processing files..."):
        post = frontmatter.loads(f.content)
        data = post.to_dict()
        data['filename'] = f.filename
        data_parsed.append(data)

    return data_parsed

In [63]:
data = parse_data(data_raw)
index = index_documents(
    data,
    chunk=True,
    chunking_params={"size": 2000, "step": 1000},
)

Output()

📄 [bold blue]Parsing documents...[/bold blue]


In [64]:
index.search(
    'How can I build an eval report with llm as a judge?',
    num_results=15
)

[{'start': 1000,
  'content': 'mply pass the selected Preset to the Report and run it over your data. If nothing else is specified, the Report will run with the default parameters for all columns in the dataset. \n\n**Single dataset**. To generate the Data Summary Report for a single dataset:\n\n```python\nreport = Report([\n    DataSummaryPreset()\n])\n\nmy_eval = report.run(eval_data_1, None)\nmy_eval\n#my_eval.json\n```\n\nAfter you `run` the Report, the resulting `my_eval` will contains the computed values for each metric, along with associated metadata and visualizations. (We sometimes refer to this computation result as a `snapshot`).\n\n<Note>\nYou can render the results in Python, export as HTML, JSON or Python dictionary or upload to the Evidently platform. Check more in [output formats](/docs/library/output_formats).\n</Note>\n\n**Two datasets**. To generate reports like Data Drift that needs two datasets, pass the second one as a reference when you `run` it:\n\n```python\nre

In [65]:
def search(query):
    return index.search(
        query=query,
        num_results=15
    )

In [66]:

instructions = """
You're an assistant that helps with the documentation.
Answer the QUESTION based on the CONTEXT from the search engine of our documentation.

Use only the facts from the CONTEXT when answering the QUESTION.

When answering the question, provide the reference to the file with the source.
Use the filename field for that.
The repo url is: https://github.com/evidentlyai/docs/

Include code examples when relevant.
If the question is discussed in multiple documents, cite all of them.

Don't use markdown or any formatting in the output.
""".strip()



In [67]:
import json
prompt_template = """
<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(question, search_results):
    context = json.dumps(search_results)

    prompt = prompt_template.format(
        question=question,
        context=context
    ).strip()

    return prompt


In [68]:
#Interact with LLM
from openai import OpenAI

openai_client = OpenAI()

def interact_with_llm(user_prompt, instructions=None, model="gpt-4o-mini"):
    messages = []

    if instructions:
        messages.append({
            "role": "system",
            "content": instructions
        })

    messages.append({
        "role": "user",
        "content": user_prompt
    })

    response = openai_client.responses.create(
        model=model,
        input=messages
    )

    return response.output_text

In [69]:
def ask_evidently(query):
    search_results = search(query)
    user_prompt = build_prompt(query,search_results)
    print(user_prompt)
    response = interact_with_llm(user_prompt,instructions)
    return response

In [70]:
result = ask_evidently('How can I build an eval report with llm as a judge?')
print(result)

<QUESTION>
How can I build an eval report with llm as a judge?
</QUESTION>

<CONTEXT>
[{"start": 1000, "content": "mply pass the selected Preset to the Report and run it over your data. If nothing else is specified, the Report will run with the default parameters for all columns in the dataset. \n\n**Single dataset**. To generate the Data Summary Report for a single dataset:\n\n```python\nreport = Report([\n    DataSummaryPreset()\n])\n\nmy_eval = report.run(eval_data_1, None)\nmy_eval\n#my_eval.json\n```\n\nAfter you `run` the Report, the resulting `my_eval` will contains the computed values for each metric, along with associated metadata and visualizations. (We sometimes refer to this computation result as a `snapshot`).\n\n<Note>\nYou can render the results in Python, export as HTML, JSON or Python dictionary or upload to the Evidently platform. Check more in [output formats](/docs/library/output_formats).\n</Note>\n\n**Two datasets**. To generate reports like Data Drift that needs 